<a href="https://colab.research.google.com/github/pacastl/PruebaIA/blob/main/Selecci%C3%B3n_de_descriptores_y_an%C3%A1lisis_estad%C3%ADstico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selección de descriptores

In [1]:
import numpy as np
import pandas as pd
# url del repositorio de github para poder leerlos
url_train = 'https://raw.githubusercontent.com/pacastl/PruebaIA/main/train.csv'
url_test = 'https://raw.githubusercontent.com/pacastl/PruebaIA/main/test.csv'

# Leemos los CSV como Pandas DataFrames
df = pd.read_csv(url_train)
#test_df = pd.read_csv(url_test)

#otra forma de leerlos porque los descargué en mi pc
train_file = r'C:\Users\Pablo\Downloads\train.csv'
test_file = r'C:\Users\Pablo\Downloads\test.csv'

#df = pd.read_csv(train_file)
#test_df = pd.read_csv(test_file)
# Mostramos los datos para comprobar que se han leído
print(df.head())
#print(test_df.head())

         SL      EEG  BP   HR  CIRCULATION  ACTIVITY
0   4019.64 -1600.00  13   79          317         3
1   2191.03 -1146.08  20   54          165         2
2   2787.99 -1263.38  46   67          224         2
3   9545.98 -2848.93  26  138          554         4
4  14148.80 -2381.15  85  120          809         4


Importamos la librería y separamos las variables predictoras y la variable objetivo:

In [2]:
from sklearn.feature_selection import SelectKBest, f_regression

X = df.drop('ACTIVITY', axis=1)
y = df['ACTIVITY']


usamos SelectKBest con f_regression como función de puntuación para seleccionar las 3 mejores

In [3]:
selector = SelectKBest(score_func=f_regression, k=3)
X_new = selector.fit_transform(X, y)


In [4]:
scores = selector.scores_
selected_features = X.columns[selector.get_support()]

print('Selected Features:', selected_features)
print('Scores:', scores[selector.get_support()])


Selected Features: Index(['SL', 'HR', 'CIRCULATION'], dtype='object')
Scores: [279.49121672 265.11169006 230.5013809 ]


Los puntajes indican la importancia de cada característica en la tarea de clasificación. En este caso, la característica con el puntaje más alto es 'SL', seguida de 'HR' y luego 'CIRCULATION'.

Se pueden utilizar las tres características para construir un modelo de clasificación, y se puede esperar que la característica 'SL' tenga el mayor impacto en la predicción del modelo

# Análisis estadístico